In [40]:
%pip install --upgrade --user google-cloud-aiplatform pymupdf rich colorama

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="./key.json"
# %system ~/google-cloud-sdk/bin/gcloud auth application-default login

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
google_project_id = os.getenv('GOOGLE_PROJECT_ID')
google_location = "europe-west1"

In [4]:
import vertexai
vertexai.init(project=google_project_id, location=google_location)

In [5]:
from IPython.display import Markdown, display
from rich.markdown import Markdown as rich_Markdown
from vertexai.generative_models import GenerationConfig, GenerativeModel, Image

text_model = GenerativeModel("gemini-1.5-pro")
multimodal_model = GenerativeModel("gemini-1.5-pro")
multimodal_model_flash = GenerativeModel("gemini-1.5-flash")


In [7]:
image_description_prompt = """Explain what is going on in the image.
If it's a table, extract all elements of the table.
If it's a graph, explain the findings in the graph.
Do not include any numbers that are not mentioned in the image.
"""

In [ ]:
from helpers import get_document_metadata

# Extract text and image metadata from the PDF document
text_metadata_df, image_metadata_df = get_document_metadata(
    multimodal_model,
    './pdf',
    image_save_dir="./images",
    image_description_prompt=image_description_prompt,
    embedding_size=1408,
    # add_sleep_after_page = True, # Uncomment this if you are running into API quota issues
    # sleep_time_after_page = 5,
    # generation_config = # see next cell
    # safety_settings =  # see next cell
)